In [ ]:
import os
import torchvision.models as models
import pandas as pd
import torch
import cv2
from PIL import Image
import random
import numpy as np
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(777)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
item_path = '/content/drive/MyDrive/DL_project/제로베이스 딥러닝 1조/IMAGE_TEST/7' # 폴더 위치
img_list = os.listdir(item_path)
img_paths = [os.path.join(item_path, f) for f in img_list]
model_path = '/content/drive/MyDrive/DL_project/제로베이스 딥러닝 1조/WEIGHT' # 각 모델 파일
save_result = '/content/drive/MyDrive/DL_project/제로베이스 딥러닝 1조/RESULT'

In [ ]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train' : transforms.Compose([
                transforms.Resize((resize, resize)),  # 이미지를 240x240로 조정
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val' : transforms.Compose([
                transforms.Resize((resize, resize)),  # 이미지를 240x240로 조정
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase):
        img = img.convert("RGB")
        return self.data_transform[phase](img)


In [ ]:
# 소분류라벨 추출
def labelExtraction(img_path):
    true_label = 10
    rank_3 = 10
    num = img_path.split('/')[-1]
    category = num[0]
    if category in ['0', '1', '2', '5', '6', '7']:
        if num.split('_')[-1][0] == '3':
            true_label = 4
            rank_3 = 4
        else:
            true_label = int(num.split('_')[1])
            rank_3 = 4
    elif category in ['3', '4', '9']:
        if num.split('_')[-1][0] == '3':
            true_label = 2
            rank_3 = 2
        else:
            true_label = int(num.split('_')[1])
            rank_3 = 2
    elif category in ['8']:
        if num.split('_')[-1][0] == '3':
            true_label = 3
            rank_3 = 3
        else:
            true_label = int(num.split('_')[1])
            rank_3 = 3


    return int(true_label) , rank_3, category

In [ ]:
class rankModel(nn.Module):
    def __init__(self, model_path, num_classes=3):
        super(rankModel, self).__init__()
        self.model = models.resnet50()
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)

        pretrained_dict = torch.load(os.path.join(model_path, 'grade_weight.pth'), map_location='cuda' if torch.cuda.is_available() else 'cpu')
        pretrained_dict = {key.replace("resnet.", ""): value for key, value in pretrained_dict.items()}
        self.model.load_state_dict(pretrained_dict)
        
    def forward(self, x):
        x = self.model(x)

        return x

In [ ]:
class loadModel(nn.Module):
    def __init__(self, category, model_path, num_classes):
        super(loadModel, self).__init__()
        self.model = models.resnet50()
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)

        pretrained_dict = torch.load(os.path.join(model_path, f'{category}_weight.pth'), map_location='cuda' if torch.cuda.is_available() else 'cpu')
        if category == '1' or 1:
            clean_pretrained_dict = {key.replace("model.", ""): value for key, value in pretrained_dict.items()}
        else:
            clean_pretrained_dict = {key.replace("resnet.", ""): value for key, value in pretrained_dict.items()}

        self.model.load_state_dict(clean_pretrained_dict)
        if torch.cuda.is_available():
            self.model = self.model.to('cuda')
        
    def forward(self, x):
        if torch.cuda.is_available():
            x = x.to('cuda')
        x = self.model(x)

        return x

In [ ]:
################# 소분류기 ###################
path_column = []
pred_column = []
label_column = []

corr_path = [] # 등급분류기 넘어 갈 것
count = 0
random.shuffle(img_paths)
with torch.no_grad():
    for img_path in img_paths:
        # 이미지 오픈
        img = Image.open(img_path)
        
        # True 라벨 추출
        true_label, rank_3, category = labelExtraction(img_path)

        mean = (0.485, 0.456, 0.406)
        std = (0.229, 0.224, 0.224)

        transform = ImageTransform(224, mean, std)
        img = transform(img, phase='val')
        img = img.unsqueeze(0)
        img = img.to(device)

        # 모델 선택
        model = loadModel(category,model_path, num_classes = (int(rank_3)+1))
        model = model.to(device)
        model.eval()
    
        outputs = model(img)

        pred = torch.argmax(outputs).item()
        pred_label = int(pred)

        path_column.append(img_path)
        pred_column.append(pred_label)
        label_column.append(true_label)


df_category = pd.DataFrame({
    'path' : path_column,
    'pred_label' : pred_column,
    'true_label' : label_column
})

df_category.sort_values(by='pred_label', inplace=True)

In [ ]:
wrong_count = len(df_category[df_category['pred_label'] != df_category['true_label']])
print(f'틀린 개수 : {wrong_count}/{len(df_category)}')

틀린 개수 : 113/400


In [ ]:
df_category['rank'] = df_category['path'].apply(lambda x: x.split('/')[-1].split('_')[-1][0])

In [ ]:
category

'7'

In [ ]:
df_category.to_csv(os.path.join(save_result, f'{category}_classification_result.csv'), index=False)

In [ ]:
corr_path = df_category[(df_category['pred_label'] == df_category['true_label']) & (df_category['pred_label'] != rank_3)]['path'].tolist()

In [ ]:
############## 등급분류기 #####################
rank_path_list = []
rank_pred_list = []
rank_label_list = []
count = 0
with torch.no_grad():
    for img_path in corr_path:
        # 이미지 오픈
        img = Image.open(img_path)
        
        # True 라벨 추출
        true_rank = int(img_path.split('/')[-1].split('_')[-1][0])
        if int(true_rank) == 3:
            count +=1
            print(f'{count} : {img_path}')
            continue

        if category == '1' or 1 or '4' or 4:
            mean = (0.429, 0.446, 0.461)
            std = (0.232, 0.238, 0.247)
        else:
            mean = (0.485, 0.456, 0.406)
            std = (0.229, 0.224, 0.224)

        transform = ImageTransform(224, mean, std)
        img = transform(img, phase='val')
        img = img.unsqueeze(0)
        img = img.to(device)

        # 모델 선택
        model = rankModel(model_path)
        model = model.to(device)
        model.eval()

        outputs = model(img)
        pred = torch.argmax(outputs).item()
        rank_pred = int(pred)
        
        rank_path_list.append(img_path)
        rank_pred_list.append(rank_pred)
        rank_label_list.append(true_rank)

In [ ]:
##### 결과 확인 ######

In [ ]:
rank_df = pd.DataFrame({
    'path' : rank_path_list,
    'pred_rank' : rank_pred_list,
    'true_rank' : rank_label_list
})

rank_df.sort_values(by='pred_rank', inplace=True)

rank_df.to_csv(os.path.join(save_result, f'{category}_rank_result.csv'), index=False)

In [ ]:
wrong_count = len(rank_df[rank_df['pred_rank'] != rank_df['true_rank']])
print(f'틀린 개수 : {wrong_count}/{len(rank_df)}')

틀린 개수 : 12/188
